In [1]:
import sys
sys.path.append('../personal_pkgs/')
import pickle
import pandas as pd
import oop_func as func
import personal_pkg as ref
import parsing_func as pars
import os
import re
from collections import Counter
import warnings
warnings.filterwarnings("ignore")
from IPython.display import Markdown

In [5]:
base_df = pd.read_csv('base_df.csv')
base_df.tail()

,author,from,index,journal,paper,year
22672,Bakshi G. and D. Madan,한국파생상품학회,19259.0,” Management Science 52 12,A theory of volatility spreads,2006.0
22673,Bakshi G. and D. Madan,한국파생상품학회,19259.0,” Management Science 52 12,A theory of volatility spreads,2006.0
22674,Bakshi G. and D. Madan,한국파생상품학회,19259.0,” Management Science 52 12,A theory of volatility spreads,2006.0
22675,Bakshi G. and D. Madan,한국파생상품학회,19259.0,” Management Science 52 12,A theory of volatility spreads,2006.0
22676,Bakshi G. and D. Madan,한국파생상품학회,19259.0,” Management Science 52 12,A theory of volatility spreads,2006.0


In [6]:
base_df['new_paper'] = \
[base_df['paper'][idx] + '////' + str(base_df['year'][idx]).split(".")[0] for idx in range(len(base_df))]

base_df['new_paper'] = base_df['new_paper'].apply(lambda x : x.lower())

base_df.tail()

,author,from,index,journal,paper,year,new_paper
22672,Bakshi G. and D. Madan,한국파생상품학회,19259.0,” Management Science 52 12,A theory of volatility spreads,2006.0,a theory of volatility spreads////2006
22673,Bakshi G. and D. Madan,한국파생상품학회,19259.0,” Management Science 52 12,A theory of volatility spreads,2006.0,a theory of volatility spreads////2006
22674,Bakshi G. and D. Madan,한국파생상품학회,19259.0,” Management Science 52 12,A theory of volatility spreads,2006.0,a theory of volatility spreads////2006
22675,Bakshi G. and D. Madan,한국파생상품학회,19259.0,” Management Science 52 12,A theory of volatility spreads,2006.0,a theory of volatility spreads////2006
22676,Bakshi G. and D. Madan,한국파생상품학회,19259.0,” Management Science 52 12,A theory of volatility spreads,2006.0,a theory of volatility spreads////2006


In [7]:
paper_ls = [i.lower() for i in base_df['new_paper'].tolist()]
counter_df = pd.DataFrame.from_dict(Counter(paper_ls),orient='index',columns=['count']).sort_values('count',ascending=False)
catch_paper_ls = counter_df[counter_df['count'] > 3].index.tolist()
display(counter_df.head())
len(catch_paper_ls)

,count
common risk factors in the returns on stocks and bonds////1993,152
on persistence in mutual fund performance////1997,97
theory of the firm: managerial behavior agency costs and ownership structure////1976,83
risk return and equilibrium: empirical tests////1973,75
the pricing of options and corporate liabilities////1973,72


939

In [8]:
catch_paper_ls[:10]

['common risk factors in the returns on stocks and bonds////1993',
 'on persistence in mutual fund performance////1997',
 'theory of the firm: managerial behavior agency costs and ownership structure////1976',
 'risk return and equilibrium: empirical tests////1973',
 'the pricing of options and corporate liabilities////1973',
 'common risk factors in the returns on bonds and stocks////1993',
 'tunneling////2000',
 'corporate financing and investment decisions when firms have information that investors do not have////1984',
 'the cross-section of expected stock returns////1992',
 'a simple positive semi-definite heteroskedasticity and autocorrelation consistent covariance matrix////1987']

- 딕셔너리로 만들어놓은 base_df에서 데이터의 산발성이 보인다.
- 따라서 catch_paper_ls 라는 딕셔너리의 딕셔너리를 구축한다. 
- 이는, base_df에서 논문의 이름과 년도를 기준으로 count를 sorting 했을 때, 3번 이상 나온 논문들만 따로 모든 것이다.
- 하지만 위에서 보듯, 
    - `'common risk factors in the returns on stocks and bonds////1993'`
    - `'common risk factors in the returns on bonds and stocks////1993'`
    - 와 같은, uniqueness 가 보장되지 않는 것을 알 수 있다. 
    - 이에 따라서, dictionary of dictionary의 uniqueness 를 보장하기 위한 작업을 아래에서 시행한다.

In [9]:
catch_paper_ls_copy = catch_paper_ls.copy()
tuned_catch_paper_ls = []
excepted_ls = []
similar_match_ls = []

for i in range(5) : 
    print(i)
    for idx1,val1 in enumerate(catch_paper_ls_copy) : 
        for idx2,val2 in enumerate(catch_paper_ls_copy) :
            if idx1 != idx2 : 
                v1 = set(val1.split(' '))
                v2 = set(val2.split(' '))
                try : 
                    if len(set(v1).intersection(v2)) / ((len(v1) + len(v2))/2) > 0.75 : 
                        similar_match_ls.append((val1,val2))
                        if len(v1) > len(v2) :
                            catch_paper_ls_copy.pop(catch_paper_ls_copy.index(val2))
                            excepted_ls.append((val2))
                        else :
                            catch_paper_ls_copy.pop(catch_paper_ls_copy.index(val1))
                            excepted_ls.append(((val1)))                    
                except : pass

len(catch_paper_ls) , len(catch_paper_ls_copy) , len(excepted_ls)

0
1
2
3
4


(939, 853, 86)

- 61개가 줄었다. 이를 기반으로 딕셔너리를 생성한다.
- ready_to_taget_dict라는 변수를 생성한다. 이는, 472개로 된 key를 가지고 있고, 이와 유사한 논문 이름들을 자기 자신을 포함해서 리스트 형태의 value를 가진다.

In [10]:
ready_to_target_dict = dict(zip(catch_paper_ls_copy,[[] for i in range(len(catch_paper_ls_copy))]))

In [11]:
append = 0
naive_append = 0

for idx1,val1 in enumerate(base_df['new_paper'].tolist()) : 
    if idx1 % 4000 == 0 : print(idx1,end='\r')
    for idx2,val2 in enumerate(ready_to_target_dict.keys()) :
        
        paper_name = val1.split('////')[0].lower()
        paper_year = val1.split('////')[1].lower()
        target_name = val2.split('////')[0].lower()
        target_year = val2.split('////')[1].lower()
        
        if paper_year != 'nan' :
            if paper_year == target_year :
                paper_name_to_word = set(paper_name.split(' '))
                target_name_to_word = set(target_name.split(' '))
                if len(paper_name_to_word.intersection(target_name_to_word)) / len(paper_name_to_word) > 0.75 : 
                    ready_to_target_dict[val2].append(val1)
                    append += 1
        else : pass
#             paper_name_to_word = set(paper_name.split(' '))
#             target_name_to_word = set(target_name.split(' '))
#             if len(paper_name_to_word.intersection(target_name_to_word)) / len(paper_name_to_word) > 0.9 : 
#                 ready_to_target_dict[val2].append(val1)
#                 naive_append += 1

In [12]:
final_ls = []
process_ls = []
error_ls = []

for idx1 , val in enumerate(base_df['new_paper']) : 
    process_ls = []
    for idx2, key_ in enumerate(list(ready_to_target_dict.keys())) : 
        if val in ready_to_target_dict.get(key_) : 
            process_ls.append(key_)

    if len(process_ls) == 1 :
        final_ls.append(process_ls[0])
    elif len(process_ls) == 0 :
        final_ls.append(val)
    else : final_ls.append(val)
        
len(final_ls) == len(base_df)

True

In [13]:
len(set(final_ls)) , len(set(base_df['new_paper']))

(12922, 13443)

In [14]:
base_df['tuned_new_paper'] = final_ls
base_df.tail()

,author,from,index,journal,paper,year,new_paper,tuned_new_paper
22672,Bakshi G. and D. Madan,한국파생상품학회,19259.0,” Management Science 52 12,A theory of volatility spreads,2006.0,a theory of volatility spreads////2006,a theory of volatility spreads////2006
22673,Bakshi G. and D. Madan,한국파생상품학회,19259.0,” Management Science 52 12,A theory of volatility spreads,2006.0,a theory of volatility spreads////2006,a theory of volatility spreads////2006
22674,Bakshi G. and D. Madan,한국파생상품학회,19259.0,” Management Science 52 12,A theory of volatility spreads,2006.0,a theory of volatility spreads////2006,a theory of volatility spreads////2006
22675,Bakshi G. and D. Madan,한국파생상품학회,19259.0,” Management Science 52 12,A theory of volatility spreads,2006.0,a theory of volatility spreads////2006,a theory of volatility spreads////2006
22676,Bakshi G. and D. Madan,한국파생상품학회,19259.0,” Management Science 52 12,A theory of volatility spreads,2006.0,a theory of volatility spreads////2006,a theory of volatility spreads////2006


In [15]:
from IPython.display import Markdown

In [16]:
display(Markdown('### 이전의 방법론'))
display(pd.DataFrame.from_dict(Counter(base_df['new_paper']),orient='index',columns=['count']).sort_values('count',ascending=False).head(10))
display(Markdown('### 개선된 방법론'))
display(pd.DataFrame.from_dict(Counter(base_df['tuned_new_paper']),orient='index',columns=['count']).sort_values('count',ascending=False).head(10))

### 이전의 방법론

,count
common risk factors in the returns on stocks and bonds////1993,152
on persistence in mutual fund performance////1997,97
theory of the firm: managerial behavior agency costs and ownership structure////1976,83
risk return and equilibrium: empirical tests////1973,75
the pricing of options and corporate liabilities////1973,72
common risk factors in the returns on bonds and stocks////1993,51
tunneling////2000,51
corporate financing and investment decisions when firms have information that investors do not have////1984,49
the cross-section of expected stock returns////1992,45
a simple positive semi-definite heteroskedasticity and autocorrelation consistent covariance matrix////1987,39


### 개선된 방법론

,count
common risk factors in the returns on the stocks and bonds////1993,231
theory of the firm : managerial behavior agency costs and ownership structure////1976,153
on persistence in mutual fund performance////1997,124
the pricing of options and corporate liabilities////1973,76
risk return and equilibrium: empirical tests////1973,75
corporate financing and investment decisions when firms have information that investors do not have////1984,64
a simple positive semi-definite heteroscedasticity and autocorrelation consistent covariance matrix////1987,62
returns to buying winners and selling losers: implications for stock market efficiency////1993,61
tunneling////2000,51
the cross section of expected stock returns////1992,51


In [18]:
paper_counter_df = pd.DataFrame.from_dict(Counter(base_df['tuned_new_paper']),orient='index',columns=['count']).sort_values('count',ascending=False)
paper_counter_df.reset_index(inplace=True)

In [19]:
paper_counter_df.to_csv('paper_counter_df.csv',index=False)

In [46]:
import pickle

In [47]:
pickle_out = open("ready_to_target_dict.pickle","wb")
pickle.dump(ready_to_target_dict, pickle_out)
pickle_out.close()

___________________________

#### 여태까지 저널에 관계없이 전체 논문에 대한 dictionary 와 paper_frequency_table 을 형성하였다. 
#### 아래에서부터는 각 저널별로 paper_frequency_table 을 생성한다. dictionary는 변하지 않는다.

In [2]:
fm_base_df = pd.read_csv('fm_base_df.csv')
f_base_df = pd.read_csv('f_base_df.csv')
sc_base_df = pd.read_csv('sc_base_df.csv')
dr_base_df = pd.read_csv('dr_base_df.csv')

In [3]:
def make_new_paper(base_df) :
    base_df['new_paper'] = \
    [base_df['paper'][idx] + '////' + str(base_df['year'][idx]).split(".")[0] for idx in range(len(base_df))]

    base_df['new_paper'] = base_df['new_paper'].apply(lambda x : x.lower())

    return base_df.loc[:,['author','journal','paper','year','from','new_paper']]

In [4]:
fm_base_df = make_new_paper(fm_base_df)
f_base_df = make_new_paper(f_base_df)
sc_base_df = make_new_paper(sc_base_df)
dr_base_df = make_new_paper(dr_base_df)

In [5]:
ready_to_target_dict = []
with (open("ready_to_target_dict.pickle", "rb")) as openfile:
    while True:
        try:
            ready_to_target_dict.append(pickle.load(openfile))
        except EOFError:
            break

In [7]:
ready_to_target_dict = ready_to_target_dict[0]
len(ready_to_target_dict.keys())

853

In [8]:
def final_process(base_df,ready_to_target_dict):
    final_ls = []
    process_ls = []
    error_ls = []

    for idx1 , val in enumerate(base_df['new_paper']) : 
        process_ls = []
        for idx2, key_ in enumerate(list(ready_to_target_dict.keys())) : 
            if val in ready_to_target_dict.get(key_) : 
                process_ls.append(key_)

        if len(process_ls) == 1 :
            final_ls.append(process_ls[0])
        elif len(process_ls) == 0 :
            final_ls.append(val)
        else : final_ls.append(val)

    if len(final_ls) == len(base_df) :
        print('Lower uniqueness Better Performance, orgin uniqueness: {} , revised uniqueness : {}'.format(len(set(base_df['new_paper'])),len(set(final_ls))))
        base_df['tuned_new_paper'] = final_ls
        return base_df
    else : raise TypeError('error')

In [9]:
fm_final_df = final_process(fm_base_df,ready_to_target_dict)
fm_final_df = final_process(fm_base_df,ready_to_target_dict)
f_final_df = final_process(f_base_df,ready_to_target_dict)
sc_final_df = final_process(sc_base_df,ready_to_target_dict)
dr_final_df = final_process(dr_base_df,ready_to_target_dict)

Lower uniqueness Better Performance, orgin uniqueness: 7528 , revised uniqueness : 7228
Lower uniqueness Better Performance, orgin uniqueness: 4042 , revised uniqueness : 3897
Lower uniqueness Better Performance, orgin uniqueness: 3139 , revised uniqueness : 3010
Lower uniqueness Better Performance, orgin uniqueness: 1739 , revised uniqueness : 1709


In [10]:
def make_counter_df_under_journal(base_df) :
    print('total length of this dataframe is {}'.format(len(base_df)))
    df1 = pd.DataFrame.from_dict(Counter(base_df['tuned_new_paper'].tolist()),orient='index',columns=['count'])
    df2 = df1.sort_values('count',ascending=False)
    return df2

In [11]:
display(Markdown('### 한국재무관리학회'))
display(make_counter_df_under_journal(fm_final_df).head(10))
display(Markdown('### 한국재무학회'))
display(make_counter_df_under_journal(f_final_df).head(10))
display(Markdown('### 한국증권학회'))
display(make_counter_df_under_journal(sc_final_df).head(10))
display(Markdown('### 한국파생상품학회'))
display(make_counter_df_under_journal(dr_final_df).head(10))

### 한국재무관리학회

total length of this dataframe is 9638


,count
theory of the firm : managerial behavior agency costs and ownership structure////1976,42
a simple positive semi-definite heteroscedasticity and autocorrelation consistent covariance matrix////1987,31
common risk factors in the returns on the stocks and bonds////1993,29
corporate financing and investment decisions when firms have information that investors do not have////1984,27
tunneling////2000,19
the cross section of expected stock returns////1992,17
determinants of corporate borrowing////1977,16
on persistence in mutual fund performance////1997,16
agency costs of free cash flow corporate finance and takeovers////1986,16
management ownership and market valuation : an empirical analysis////1988,15


### 한국재무학회

total length of this dataframe is 4959


...

total length of this dataframe is 5685


,count
common risk factors in the returns on the stocks and bonds////1993,136
theory of the firm : managerial behavior agency costs and ownership structure////1976,101
on persistence in mutual fund performance////1997,90
risk return and equilibrium: empirical tests////1973,50
the pricing of options and corporate liabilities////1973,48
detecting long-run abnormal stock returns: the empirical power and specification of test statistics////1997,37
tunneling or value added? evidence from mergers by korean business grups////2002,37
returns to buying winners and selling losers: implications for stock market efficiency////1993,27
trading is hazardous to your wealth: the common stock investment performance of individual investors////2000,27
the cross-section of volatility and expected returns////2006,26


### 한국파생상품학회

total length of this dataframe is 2395


,count
common risk factors in the returns on the stocks and bonds////1993,40
stock index futures////2006,25
studies of stock market volatility changes////1976,21
results////2007,16
a theory of volatility spreads////2006,16
asset pricing////2001,16
stock return characteristics skew laws and the differential pricing of individual equity options////2003,13
sd////2000,13
kospi200 옵션시장에서의 박스스프레드 차익거래 수익성////2006,13
a simple positive semi-definite heteroscedasticity and autocorrelation consistent covariance matrix////1987,13
